# PART ONE

In [1]:
# Import Statements

import pandas as pd

#### Pre-processing in the cell below:
* Read in table from Wikipedia
* Remove rows where no Borough was assigned
* Rename rows where the Neighbourhood had no name to the Borough in which they reside
* Merged rows which shared a postcode

In [2]:
# Wikipedia Scraping and data pre-processing

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # read Wiki
TN_df = tables[0]                                                                        # select desired table
TN_df = TN_df[TN_df.Borough != "Not assigned"]                                           # remove undesired rows

                                 # adjust Not Assigned neighbourhoods to Borough name
for x in range(0,TN_df.shape[0]):
    if TN_df.iloc[x].Neighbourhood == "Not assigned":
        TN_df.iloc[x].Neighbourhood = TN_df.iloc[x].Borough
        
                           # group by Postcode and aggregate Neighbourhoods with ', ' in between
                           # reset index to numbering
TN_df = TN_df.groupby(TN_df['Postcode']).agg({'Borough':'first','Neighbourhood':', '.join})
TN_df.reset_index(inplace=True)
        
# TN_df.describe() # used to ensure unique values for each postcode
TN_df.shape

(103, 3)

In [3]:
TN_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# PART TWO

#### Attempted to use geocoder, but that did not work. Utilizing the csv provided in the assignment, I did the following:
* Loaded the data from the CSV
* Merged the dataframes using the postcode
* Dropped the redundant postcode column

In [146]:
TN_coords = pd.read_csv('https://cocl.us/Geospatial_data')
TN_coords.head()

TN_full = TN_df.merge(TN_coords, left_on='Postcode', right_on='Postal Code')
TN_full.drop(['Postal Code'],axis=1,inplace=True)
TN_full.head()
#TN_full.shape

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# PART THREE

In [5]:
# @hidden cell
CLIENT_ID = 'IZAFYENE22QRBKRCBZPGCMOBNBJP0NI5QWW4VAGBZXFXLSPG'
CLIENT_SECRET = 'XZS5WIFHHMKO3HCQK4NV4TGVSDX4LBLMKWTRUXGWI15ZB3VJ'
VERSION = '20180605'

In [51]:
# Import statements

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

#### Defined the function from week 3's lab for retrieving venues around a location and compiling them into a dataframe

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=100):
    i = 1
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(i, ': ', name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        i = i+1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### I decided to retrieve venues for all the neighborhoods - some will be dropped due to lack of venues

In [53]:
toronto_venues = getNearbyVenues(names=TN_full['Neighbourhood'],
                                 latitudes=TN_full['Latitude'],
                                 longitudes=TN_full['Longitude'])

1 :  Rouge, Malvern
2 :  Highland Creek, Rouge Hill, Port Union
3 :  Guildwood, Morningside, West Hill
4 :  Woburn
5 :  Cedarbrae
6 :  Scarborough Village
7 :  East Birchmount Park, Ionview, Kennedy Park
8 :  Clairlea, Golden Mile, Oakridge
9 :  Cliffcrest, Cliffside, Scarborough Village West
10 :  Birch Cliff, Cliffside West
11 :  Dorset Park, Scarborough Town Centre, Wexford Heights
12 :  Maryvale, Wexford
13 :  Agincourt
14 :  Clarks Corners, Sullivan, Tam O'Shanter
15 :  Agincourt North, L'Amoreaux East, Milliken, Steeles East
16 :  L'Amoreaux West
17 :  Upper Rouge
18 :  Hillcrest Village
19 :  Fairview, Henry Farm, Oriole
20 :  Bayview Village
21 :  Silver Hills, York Mills
22 :  Newtonbrook, Willowdale
23 :  Willowdale South
24 :  York Mills West
25 :  Willowdale West
26 :  Parkwoods
27 :  Don Mills North
28 :  Flemingdon Park, Don Mills South
29 :  Bathurst Manor, Downsview North, Wilson Heights
30 :  Northwood Park, York University
31 :  CFB Toronto, Downsview East
32 :  Downs

#### 3 Neighborhoods had no venues

In [54]:
count_venues = toronto_venues.shape[0]
count_neighborhood = toronto_venues.groupby('Neighbourhood').count().shape[0]
count_unique_categories = len(toronto_venues['Venue Category'].unique())
print("Venues: {}, Neighbourhoods: {}, Unique Categories: {}".format(count_venues, count_neighborhood, count_unique_categories))

Venues: 2248, Neighbourhoods: 100, Unique Categories: 273


#### When I originally checked the count, I found 52 neighborhoods had less than 10 venues
#### I decided that these would not be useful, and dropped them:
* Got the names of the neighbourhoods which had less than 10 venues
* Created a new dataframe with those neighborhoods dropped

In [107]:
venues_counted = toronto_venues.groupby('Neighbourhood').count() 
#   52 neighbourhoods with less than 10 venues

#   get neighbourhood names for neighbourhoods with less than 10 venues
lessthanten_names = venues_counted[ venues_counted['Venue'] < 10].index
lessthanten_names.shape

#   isin() returns true if the value is in the list. I want to remove those values, so adding ~ effectively drops those rows
toronto_venues_trimmed = toronto_venues[~toronto_venues['Neighbourhood'].isin(lessthanten_names)]
toronto_venues_trimmed.reset_index(inplace = True)
toronto_venues_trimmed.shape

#toronto_venues_trimmed.groupby('Neighbourhood').describe(include='all')    # used to check # of unique categories after trimming
#   testing this trimming method resulted in neighbourhoods having a minimum of 9 unique venue categories, which is better than having neighbourhoods with fewer than 5 total venues

(2028, 8)

#### One-hot encoding of the venues based on venue category

In [108]:
toronto_onehot = pd.get_dummies(toronto_venues_trimmed[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues_trimmed['Neighbourhood']                             # had to use Canadian spelling due to venue category 'Neighborhood' existing
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Clarks Corners, Sullivan, Tam O'Shanter",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [109]:
toronto_onehot.shape

(2028, 250)

#### Grouped by Neighbourhood, and averaged the one-hot encoding for frequency

In [110]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape

(48, 250)

#### Created dataframe listing the most common venue categories

In [111]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10
indicators = ['st','nd','rd']

columns=['Neighbourhood']
for index in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(index+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(index+1))
        
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for index in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[index, 1:] = return_most_common_venues(toronto_grouped.iloc[index, :], num_top_venues)
    
toronto_venues_sorted.head()

,Neighbourhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Asian Restaurant,Cosmetics Shop,Breakfast Spot,Hotel,Restaurant,American Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Gym,American Restaurant
2,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sandwich Place,Bridal Shop,Deli / Bodega,Supermarket,Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Liquor Store,Sandwich Place,Juice Bar,Fast Food Restaurant,Butcher,Restaurant
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Italian Restaurant,Bakery,Beer Bar,Steakhouse


#### Ran K-Means clustering on the data
I tried with 4, 5, and 6 clusters. 6 clusters rendered the most useful looking clustering

In [135]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([3, 2, 3, 2, 3, 3, 0, 3, 3, 1], dtype=int32)

#### Merged original data table with the venue category table and new cluster labels list

In [136]:
try:
    toronto_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
except:
    toronto_venues_sorted['Cluster Labels'] = kmeans.labels_
    
toronto_merged = TN_full

toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how='right')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,4,Pizza Place,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Thai Restaurant,Noodle House,Fried Chicken Joint,Coworking Space
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,2,Fast Food Restaurant,Chinese Restaurant,Gym,Coffee Shop,Cosmetics Shop,Sandwich Place,Bubble Tea Shop,Breakfast Spot,Pharmacy,Pizza Place
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Sporting Goods Shop,Asian Restaurant,Bus Station,Bakery,Japanese Restaurant,Electronics Store
22,M2N,North York,Willowdale South,43.770120,-79.408493,3,Coffee Shop,Ramen Restaurant,Sushi Restaurant,Pizza Place,Café,Restaurant,Japanese Restaurant,Sandwich Place,Shopping Mall,Indonesian Restaurant
27,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,3,Coffee Shop,Asian Restaurant,Beer Store,Gym,Sandwich Place,Bike Shop,Restaurant,Italian Restaurant,Dim Sum Restaurant,Japanese Restaurant


#### Displayed the map

In [145]:
latitude = 43.668477
longitude = -79.389994
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill = True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

#### Listed the clusters to find the relationship between clustered neighbourhoods

My guess:
* Cluster 0 : Cafe
* Cluster 1 : Coffee Shop / "Gym/Fitness Center"
* Cluster 2 : Fast Food / Indian / Sandwich Place
* Cluster 3 : Coffee Shop / Cafe
* Cluster 4 : Pizza Place
* Cluster 5 : Skating Rink

In [138]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,0,Café,Bookstore,Restaurant,Japanese Restaurant,Bar,Bakery,Comfort Food Restaurant,French Restaurant,Sandwich Place,Dessert Shop
67,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Dumpling Restaurant,Bakery,Vietnamese Restaurant,Mexican Restaurant,Coffee Shop,Donut Shop
75,Downtown Toronto,0,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Nightclub,Diner,Baby Store,Convenience Store,Coffee Shop
76,West Toronto,0,Pharmacy,Bakery,Supermarket,Park,Music Venue,Café,Middle Eastern Restaurant,Brewery,Bar,Bank
77,West Toronto,0,Bar,Coffee Shop,Asian Restaurant,Cocktail Bar,Men's Store,Restaurant,Café,Pizza Place,Bakery,New American Restaurant
82,West Toronto,0,Café,Mexican Restaurant,Fast Food Restaurant,Fried Chicken Joint,Grocery Store,Music Venue,Diner,Cajun / Creole Restaurant,Discount Store,Bowling Alley
87,East Toronto,0,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Comic Shop,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
92,Etobicoke,0,Hardware Store,Bakery,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Burrito Place,Burger Joint,Sandwich Place,Supplement Shop


In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Mississauga,1,Coffee Shop,Hotel,Gym / Fitness Center,Middle Eastern Restaurant,Sandwich Place,Burrito Place,Fried Chicken Joint,American Restaurant,Mediterranean Restaurant,Donut Shop


In [140]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Scarborough,2,Fast Food Restaurant,Chinese Restaurant,Gym,Coffee Shop,Cosmetics Shop,Sandwich Place,Bubble Tea Shop,Breakfast Spot,Pharmacy,Pizza Place
39,East York,2,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Liquor Store,Discount Store,Park,Pharmacy,Pizza Place,Coffee Shop
42,East Toronto,2,Park,Pizza Place,Fast Food Restaurant,Italian Restaurant,Burrito Place,Fish & Chips Shop,Movie Theater,Ice Cream Shop,Steakhouse,Pub
47,Central Toronto,2,Sandwich Place,Dessert Shop,Pharmacy,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Thai Restaurant
62,North York,2,Coffee Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Liquor Store,Sandwich Place,Juice Bar,Fast Food Restaurant,Butcher,Restaurant
65,Central Toronto,2,Sandwich Place,Coffee Shop,Café,Pizza Place,BBQ Joint,Park,Jewish Restaurant,Indian Restaurant,Pub,Burger Joint
88,Etobicoke,2,Café,Gym,Fried Chicken Joint,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Mexican Restaurant,Fast Food Restaurant
89,Etobicoke,2,Pizza Place,Coffee Shop,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Gym,American Restaurant


In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,North York,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Sporting Goods Shop,Asian Restaurant,Bus Station,Bakery,Japanese Restaurant,Electronics Store
22,North York,3,Coffee Shop,Ramen Restaurant,Sushi Restaurant,Pizza Place,Café,Restaurant,Japanese Restaurant,Sandwich Place,Shopping Mall,Indonesian Restaurant
27,North York,3,Coffee Shop,Asian Restaurant,Beer Store,Gym,Sandwich Place,Bike Shop,Restaurant,Italian Restaurant,Dim Sum Restaurant,Japanese Restaurant
28,North York,3,Coffee Shop,Sandwich Place,Bridal Shop,Deli / Bodega,Supermarket,Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Fried Chicken Joint,Frozen Yogurt Shop
38,East York,3,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Smoothie Shop,Breakfast Spot,Sushi Restaurant,Mexican Restaurant,Supermarket,Fish & Chips Shop
41,East Toronto,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Caribbean Restaurant,Bookstore,Brewery,Bubble Tea Shop
43,East Toronto,3,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Bookstore,Convenience Store,Diner,Latin American Restaurant,Chinese Restaurant
46,Central Toronto,3,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Ice Cream Shop,Park,Spa,Mexican Restaurant,Diner,Burger Joint
49,Central Toronto,3,Coffee Shop,Pub,Light Rail Station,American Restaurant,Restaurant,Bagel Shop,Supermarket,Liquor Store,Fried Chicken Joint,Sports Bar
51,Downtown Toronto,3,Coffee Shop,Pizza Place,Park,Bakery,Restaurant,Café,Grocery Store,Pub,Italian Restaurant,Japanese Restaurant


In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,4,Pizza Place,Pharmacy,Chinese Restaurant,Fast Food Restaurant,Bank,Italian Restaurant,Thai Restaurant,Noodle House,Fried Chicken Joint,Coworking Space
35,East York,4,Fast Food Restaurant,Pizza Place,Gastropub,Gym / Fitness Center,Café,Bank,Intersection,Athletics & Sports,Pharmacy,Gay Bar


In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,East York,5,Skating Rink,Spa,Cosmetics Shop,Pharmacy,Beer Store,Curling Ice,Athletics & Sports,Video Store,Park,Comic Shop
